# Environment Configurator

The purpose of this notebook is to ensure we have a standardized base configuration.

This notebook can be used to download the required PowerShell modules.

In [9]:
# Review the working folder path
$pwd

# Review the PPROFILE path (the PROFILE is executed every time a Notebook is loaded)
$PROFILE.CurrentUserCurrentHost

$env:PSModulePath.Split([System.IO.Path]::PathSeparator)


Path
----
/Users/sass/Source/SwitchPwshXDRNotebooks
/Users/sass/.config/powershell/Microsoft.dotnet-interactive_profile.ps1
/Users/sass/Source/SwitchPwshXDRNotebooks/Modules
/Users/sass/.local/share/powershell/Modules
/Users/sass/.nuget/packages/microsoft.dotnet-interactive/1.0.617701/tools/net9.0/any/Modules
/Users/sass/.nuget/packages/microsoft.dotnet-interactive/1.0.617701/tools/net9.0/any/runtimes/unix/lib/net8.0/Modules
/Users/sass/Source/SwitchPwshXDRNotebooks/Modules
/usr/local/share/powershell/Modules



## Code to be added to your profile

**DO NOT MODIFY THIS**

In [6]:
# calculate the path to the Modules folder in the current project folder where the notebook is located
$modulePath = Join-Path $pwd "Modules"

function findGitignore {
    # this function will try to locate the .gitignore file in the current folder or any parent folder
    [CmdletBinding()]
    param (
        [Parameter(Mandatory=$true)]
        [ValidateNotNullOrEmpty()]
        [string]$path
    )
    Write-Verbose "Checking $path"
    if ( Get-ChildItem $path ".gitignore" -Force -ErrorAction SilentlyContinue ) {
        $error.Clear()
        return $path
    } else {
        findGitignore $(Split-Path $path)
    }
}

# checking that the .gitignore file exists in the current folder or any parent folder
try { $projectRoot = findGitignore -path $modulePath -Verbose } catch {}
Write-Host "Project root: $projectRoot"

# if the .gitignore file is found, then add the Modules folder to the PSModulePath
if ($null -ne $projectRoot) {
    $modulePath = Join-Path $projectRoot "Modules"
    # create the Modules folder if it does not exist
    if (-not (Test-Path -Path $modulePath)) {
        New-Item -Path $modulePath -ItemType Directory -Force | Out-Null
    }
    # and we will also remove any duplicates and OneDrive paths
    $env:PSModulePath = ($env:PSModulePath.Split([System.IO.Path]::PathSeparator) | Sort-Object -Unique | Where-Object {-not $_.Contains("OneDrive")}) -join [System.IO.Path]::PathSeparator
    
    # Add the $modulePath folder to the PSModulePath
    # we will add it to the front of the PSModulePath so that it will have priority over any other modules with the same name
    # this is useful for development purposes, so that we can test the modules without having to
    # install them globally or in the user profile
    $env:PSModulePath = "$modulePath$([System.IO.Path]::PathSeparator)$env:PSModulePath"
}

# setting the default value for the Connect-MgGraph:ContextScope parameter to 'Process' so Notebooks won't share the same context
$PSDefaultParameterValues["Connect-MgGraph:ContextScope"] = 'Process'

VERBOSE: Checking /Users/sass/Source/SwitchPwshXDRNotebooks/Modules
VERBOSE: Checking /Users/sass/Source/SwitchPwshXDRNotebooks
Project root: /Users/sass/Source/SwitchPwshXDRNotebooks


## Modules to be downloaded

In [10]:
$requiredModules = @(
    'PowerShellNotebook',
    'Microsoft.Graph.Authentication',
    'Microsoft.Graph.Users',
    'Microsoft.Graph.Groups',
    'Microsoft.Graph.Applications',
    'Microsoft.Graph.DeviceManagement',
    'Microsoft.Graph.Security'
)

$modules = Find-Module -Name $requiredModules -ErrorAction SilentlyContinue
$modules.count
#Save-Module $requiredModules -path $modulePath -Force

7


Error: Access to the path '/usr/local/share/powershell' is denied.
Access to the path '/usr/local/share/powershell' is denied.

## PROFILE content enforcement

In [8]:
# drop the signed profile into the current user's profile folder

Copy-Item -Path "$pwd\$(Split-Path $PROFILE.CurrentUserCurrentHost -Leaf)" -Destination $PROFILE.CurrentUserCurrentHost -Force

In [11]:
# checking the content of the PROFILE file

Get-Content $PROFILE.CurrentUserCurrentHost # | select -First 20

# calculate the path to the Modules folder in the current project folder where the notebook is located
$modulePath = Join-Path $pwd "Modules"

function findGitignore {
    # this function will try to locate the .gitignore file in the current folder or any parent folder
    [CmdletBinding()]
    param (
        [Parameter(Mandatory=$true)]
        [ValidateNotNullOrEmpty()]
        [string]$path
    )
    Write-Verbose "Checking $path"
    if ( Get-ChildItem $path ".gitignore" -Force -ErrorAction SilentlyContinue ) {
        $error.Clear()
        return $path
    } else {
        findGitignore $(Split-Path $path)
    }
}

# checking that the .gitignore file exists in the current folder or any parent folder
try { $projectRoot = findGitignore -path $modulePath -Verbose } catch {}
Write-Host "Project root: $projectRoot"

# if the .gitignore file is found, then add the Modules folder to the PSModulePath
if ($null -ne $projectRoot) {
    $modulePath = Join-Path $projectRoot "Modules"
    # 